High-performance and parallel computing for AI - Practical 3: Vectorization/Tensorization
=========================================================================================

IMPORTANT
=========

* For these practicals we will be using a different `conda environment`. When opening a notebook or a terminal make sure you are using the **CuPy Kernel**!!!
* It's fine if you do not finish everything.

Question 1
----------

For $n=4096$, let $a,b\in\mathbb{R}^n$ be vectors with random entries. Consider the following operations:

1- Entrywise product $a*b$.

2- Outer product $ab^T$.

For each of these, write and time two functions, the first using pure Python for loops over the array entries, the latter using numpy vector/matrix operations. Which implementation is the fastest? Why?

Solution to Question 1
----------------------

In [1]:
import numpy as np
from time import time

n = 4096
    
a = np.random.randn(n)
b = np.random.randn(n)

c = np.zeros((n,))
C = np.zeros((n, n))

tic = time()
for i in range(n):
    c[i] = a[i]*b[i]
toc = time()

print("Pure Python entrywise product:", toc-tic)

tic = time()
c = a*b
toc = time()

print("Numpy entrywise product:", toc-tic)

tic = time()
for i in range(n):
    for j in range(n):
        C[i,j] = a[i]*b[j]

toc = time()
    
print("Pure Python outer product:", toc-tic)

tic = time()
C = np.outer(a,b)
toc = time()
print("Numpy outer product:", toc-tic)

Pure Python entrywise product: 0.001995086669921875
Numpy entrywise product: 0.0004477500915527344
Pure Python outer product: 4.832041263580322
Numpy outer product: 0.024042844772338867


Question 2
----------

The code snippets below do not use vectorization. Can you write the corresponding vectorized code using numpy only and check that the answers match? Time both versions (vectorized or not) and check the vectorized code is faster.

In [2]:
import numpy as np

N = 128

A = np.arange(N*N).reshape((N,N))
a = np.arange(N)

######### Code snippet 1 ############
x = np.array([i+j for (i,j) in zip(range(N), range(N))])

######### Code snippet 2 ############
x = np.array([[i+j for i in range(N)] for j in range(N)])

######### Code snippet 3 ############
C = np.zeros((N,N))

for i in range(N):
    for j in range(N):
        for k in range(N):
            C[i,k] += A[i,j]*A[k,j]

######### Code snippet 4 ############
C = np.zeros((N,N))

for i in range(N):
    for j in range(N):
        C[i,j] += np.cos(a[i]) + np.sin(a[j])

######### Code snippet 5 ############
z = np.random.rand(N*N)
c = np.zeros((N*N,))

for i in range(N*N):
    c[i] = 3 if z[i] > 0.5 else -1

######### Code snippet 6 ############

# Hint: use numpy.einsum

C = np.zeros((N,N,N))

for i in range(N):
    for j in range(N):
        for k in range(N):
            C[i,j,k] += A[i,j]*a[k]


Solution to Question 2
----------------------

In [3]:
import numpy as np
from time import time

N = 128

A = np.arange(N*N).reshape((N,N))
a = np.arange(N)

t = np.zeros((3,))

######### Code snippet 1 ############
t[0] = time()
x = np.array([i+j for (i,j) in zip(range(N), range(N))])
t[1] = time()
y = 2*a
t[2] = time()
assert np.allclose(x,y)
print("Snippet 1, timings: (%.3e, %.3e)" % tuple(t[1:] - t[:-1]))

######### Code snippet 2 ############
t[0] = time()
x = np.array([[i+j for i in range(N)] for j in range(N)])
t[1] = time()
y = a + a.reshape((-1,1))
t[2] = time()
assert np.allclose(x,y)
print("Snippet 2, timings: (%.3e, %.3e)" % tuple(t[1:] - t[:-1]))

######### Code snippet 3 ############
C = np.zeros((N,N))
D = C.copy()

t[0] = time()
for i in range(N):
    for j in range(N):
        for k in range(N):
            C[i,k] += A[i,j]*A[k,j]
t[1] = time()
D = A@A.T
t[2] = time()
assert np.allclose(C,D)
print("Snippet 3, timings: (%.3e, %.3e)" % tuple(t[1:] - t[:-1]))

######### Code snippet 4 ############
C = np.zeros((N,N))
D = C.copy()

t[0] = time()
for i in range(N):
    for j in range(N):
        C[i,j] += np.cos(a[i]) + np.sin(a[j])
t[1] = time()
D = np.cos(a).reshape((-1,1)) + np.sin(a)
t[2] = time()
assert np.allclose(C,D)
print("Snippet 4, timings: (%.3e, %.3e)" % tuple(t[1:] - t[:-1]))

######### Code snippet 5 ############
z = np.random.rand(N*N)
c = np.zeros((N*N,))

t[0] = time()
for i in range(N*N):
    c[i] = 3 if z[i] > 0.5 else -1
t[1] = time()
d = 4*(z>0.5)-1
t[2] = time()
assert np.allclose(c,d)
print("Snippet 5, timings: (%.3e, %.3e)" % tuple(t[1:] - t[:-1]))

######### Code snippet 6 ############
C = np.zeros((N,N,N))

t[0] = time()
for i in range(N):
    for j in range(N):
        for k in range(N):
            C[i,j,k] += A[i,j]*a[k]

t[1] = time()
D = np.einsum('ij,k->ijk', A, a)
t[2] = time()
assert np.allclose(C,D)
print("Snippet 6, timings: (%.3e, %.3e)" % tuple(t[1:] - t[:-1]))


Snippet 1, timings: (8.750e-05, 7.153e-05)
Snippet 2, timings: (1.007e-03, 7.296e-05)
Snippet 3, timings: (9.798e-01, 1.421e-03)
Snippet 4, timings: (4.130e-02, 8.321e-05)
Snippet 5, timings: (3.125e-03, 7.677e-05)
Snippet 6, timings: (9.806e-01, 3.690e-03)


Question 3
----------

The code snippets below do not use vectorization. Given what we have seen in the lectures are they suitable for vectorization?

JIT-compile the code below using numba so that the compiled code uses as much vectorization as it can. ONLY USE OPERATIONS THAT ACT ON SINGLE ENTRIES. Then, time each code snippet.

Can you modify the code so that each function is more amenable to vectorization? If so, do it and check the timings again. Did you make them faster?

Hint: All functions can be modified and made faster with the exception of snippet 4. Snippet 4 can be modified so that it runs faster, but in practice it does not seem to matter.

Advice: Always check that each function yields the same exact numbers after modification.

In [4]:
import numpy as np
from numba import jit
from time import time

N = 2**14

a = np.arange(N)
b = np.random.permutation(a)

t = np.zeros((3,))

######### Code snippet 1 ############

def snippet1(a):
    out = np.zeros_like(a)
    for i in range(N):
        if i > 100 and i < 110:
            out[i] = 30 if i == 101 else 45
        else:
            out[i] = np.sqrt(i)

    return out

c = snippet1(a)
            
######### Code snippet 2 ############

# Hint: use the fact that a[i] = i

def snippet2(a):
    out = np.zeros_like(a)
    for i in range(N):
        out[i] = np.sin(i)*a[i]
        if np.log(a[i]+1) >= np.log(N//2):
            break

    return out

c = snippet2(a)

######### Code snippet 3 ############

# Hint: use the fact that a[i] = i

def snippet3(a,b):
    out = np.zeros_like(a)
    for i in range(N):
        out[i] = a[b[i]]

    return out

c = snippet3(a,b)

######### Code snippet 4 ############

# Note: This snippet will likely yield the same timings.
#       Let us comment it in class.

@jit('float64(float64)', nopython=False)
def myfun(x):
    return np.exp(np.log(x+1))

def snippet4(a):
    out = np.zeros_like(a)
    for i in range(N):
        out[i] = myfun(a[i])

    return out

c = snippet4(a)

Solution to Question 3
----------------------

In [5]:
######### Code snippet 1 ############

@jit
def fast1(a):
    out = np.zeros_like(a)
    for i in range(N):
        out[i] = np.sqrt(i)
        
    out[101] = 30
    
    for i in range(102,110):
        out[i] = 45
    
    return out
            
######### Code snippet 2 ############

@jit
def fast2(a):
    out = np.zeros_like(a)
    for i in range(N//2):
        out[i] = np.sin(i)*a[i]
    return out

######### Code snippet 3 ############

@jit
def fast3(a,b):
    out = np.zeros_like(a)
    for i in range(N):
        out[i] = b[i]

    return out

######### Code snippet 4 ############

@jit
def fast4(a):
    out = np.zeros_like(a)
    for i in range(N):
        out[i] = np.exp(np.log(a[i]+1))

    return out

########################################################

snips = [snippet1, snippet2, snippet3, snippet4]
fsnips = [fast1, fast2, fast3, fast4]
jsnips = []

# Call everything once to trigger JIT before we time things
for i in range(len(snips)):
    snip = jit(snips[i])
    jsnips.append(snip)
    cj = jsnips[i](a) if i != 2 else jsnips[i](a,b)
    cf = fsnips[i](a) if i != 2 else fsnips[i](a,b)
    
for i in range(len(snips)):
    t[0] = time()
    for j in range(500):
        cj = jsnips[i](a) if i != 2 else jsnips[i](a,b)
    t[1] = time()
    for j in range(500):
        cf = fsnips[i](a) if i != 2 else fsnips[i](a,b)
    t[2] = time()

    assert np.allclose(cj,cf)
    print("Snip %d, timings: (%.3e, %.3e)" % (i+1, t[1]-t[0], t[2]-t[1]))

Snip 1, timings: (2.166e-02, 1.030e-02)
Snip 2, timings: (1.250e-01, 7.605e-02)
Snip 3, timings: (6.559e-03, 2.376e-03)
Snip 4, timings: (9.221e-02, 9.263e-02)


Question 4
----------

This question is designed to make you understand how AVX vector units are actually used by compilers.

Let $a,b\in\mathbb{R}^n$ be two vectors with random entries. The objective is to write a Python function that implements the vector dot product $a\cdot b$ by mimicking what a compiler does.

**Catch**: In what follows you are only allowed to use either scalar operations on entries or the functions provided below. You are not allowed to use numba other than what is already provided nor you are allowed to modify the provided functions.

Write a function `dotprod` that implements $a\cdot b$ for $n=1024$ by following these steps:

1- Read and understand what the provided functions do. You can choose to use either single or double precision arrays in the excercise. **Hint:** If you use single precision keep in mind that numpy uses double by default so you need to pass `dtype=np.float32`, e.g., `np.zeros((16,), dtype=np.float32)`.

2- How can you write a dot product using the AVX functions? **Hint:** Let $M=8$ for double and $M=16$ for single. Loop over size $M$ blocks of the inputs and accumulate the result in a single length-$M$ vector. After the loop, simply add these $M$ entries together.

3- Check that your code is correct by comparing the results with what obtained by using `@` in numpy.

Once you are done, repeat the excercise for $n=1031$. Now $n$ is not a multiple of $M$ so you have two options:

a) **Padding.** Pad the vectors with zeros so that they have a size divisible by $M$.

b) **Remainder loop.** Use a for loop to sum over the remaining $(n$ mod $M)$ entries.

Implement both options in two functions `dotprod_pad` and `dotprod_rem`.

In [6]:
import numpy as np
from numba import jit, float64, float32

f64 = float64[:](float64[:], float64[:], float64[:])
f32 = float32[:](float32[:], float32[:], float32[:])

@jit(f64)
def AVX512d(a,b,c):
    assert a.size == 8 and b.size == 8 and c.size == 8
    d = np.zeros((8,))
    for i in range(8):
        d[i] = a[i]*b[i] + c[i]

    return d

@jit(f32)
def AVX512s(a,b,c):
    assert a.size == 16 and b.size == 16 and c.size == 16
    d = np.zeros((16,), dtype=np.float32)
    for i in range(16):
        d[i] = a[i]*b[i] + c[i]

    return d

# NOTE: AVX512d only accepts double-precision vectors of size 8.
#       AVX512s only accepts single-precision vectors of size 16.
#       This is to mimic the way AVX512 units work (512 bits is the memory occupied by 8 doubles or 16 singles).

# Example usage (also for first-time JIT compiling)
a = np.random.randn(8)
b = np.random.randn(8)
c = np.random.randn(8)

d = AVX512d(a,b,c)

a = np.random.randn(16).astype(np.float32)
b = np.random.randn(16).astype(np.float32)
c = np.random.randn(16).astype(np.float32)

d = AVX512s(a,b,c)

dtype = np.float32 # or np.float64, you choose

M = 8 if isinstance(dtype, np.float64) else 16

AVX = AVX512d if isinstance(dtype, np.float64) else AVX512s # This way you can simply use the function AVX

n = 1024

a = np.random.randn(n).astype(dtype)
b = np.random.randn(n).astype(dtype)

Solution to Question 4
----------------------

In [7]:
n = 1031
dtype = np.float32 # or np.float64, you choose

a = np.random.randn(n).astype(dtype)
b = np.random.randn(n).astype(dtype)

M = 8 if isinstance(dtype, np.float64) else 16

AVX = AVX512d if isinstance(dtype, np.float64) else AVX512s

def dotprod_rem(a,b):
    
    assert b.size == a.size
    
    n = a.size

    d = np.zeros((M,), dtype=dtype)
    nblocks = n//M # integer division
    rem = n%M # taking the mod

    for i in range(nblocks):
        d = AVX(a[i*M:(i+1)*M], b[i*M:(i+1)*M], d)

    out = 0.
    for i in range(M):
        out += d[i]
    
    for j in range(rem):
        out += a[nblocks*M + j]*b[nblocks*M + j]

    return out

def dotprod_pad(a,b):
    
    assert b.size == a.size
    
    n = a.size

    d = np.zeros((M,), dtype=dtype)
    nblocks = n//M # integer division
    rem = n%M # taking the mod

    # Need to pad and copy values
    if rem > 0:
        nn = (nblocks+1)*M
        aa = np.zeros((nn,), dtype=dtype)
        bb = np.zeros((nn,), dtype=dtype)
        for i in range(n):
            aa[i] = a[i]
            bb[i] = b[i]

        a = aa
        b = bb
        n = nn
        nblocks += 1

    for i in range(nblocks):
        d = AVX(a[i*M:(i+1)*M], b[i*M:(i+1)*M], d)

    out = 0.
    for i in range(M):
        out += d[i]
    
    return out

exact = a@b
outrem = dotprod_rem(a,b)
outpad = dotprod_pad(a,b)
assert np.allclose(exact, outpad)
assert np.allclose(outrem, exact)

Question 5
----------

This question is designed to make you understand how matrix units could be used by compilers (unlike for AVX units, compilers do not use tensorization automatically at the moment).

Let $A\in\mathbb{R}^{m \times k}$, $B\in\mathbb{R}^{k \times n}$ be two rectangular matrices with random entries. The objective is to write a Python function that implements the matrix product $AB$. 

**Catch**: In what follows you are only allowed to use either scalar operations on entries or the functions provided below. You are not allowed to use numba other than what is already provided nor you are allowed to modify the provided functions. However, here you are allowed to use array slicing and array assignment.

Write a function `matmat` that implements $AB$ for $m=32$, $n=16$, $k=1024$ by following these steps:

1- Read and understand what the provided functions do. Here you must use single precision since matrix units are most efficient at low precisions.

2- How can you write a matrix-matrix product using the provided functions? **Hint:** Block size here is $M=n$. Loop over size $M\times M$ blocks of the inputs and accumulate the result in two $M\times M$ matrices.

3- Check that your answers are correct by performing the matrix multiplication using `@`.

Once you are done, think about how you would implement the matrix product under the following scenarios (only think, do not do it, unless you have time):

a- $m=64$, $n=32$.

b- $m$, $n$, and $k$ are not multiples of $16$.

c- Finally, think about how matrix units could also be used to compute the dot product between a bunch of different vectors at the same time (this is called batching).

In [8]:
import numpy as np
from numba import jit, float32

f32 = float32[:,:](float32[:,:], float32[:,:], float32[:,:])

@jit(f32)
def MU(A,B,C):
    assert A.shape[0] == A.shape[1] and B.shape[0] == B.shape[1] and C.shape[0] == C.shape[1]
    assert A.size == B.size and B.size == C.size
    D = np.zeros((16,16), dtype=np.float32)
    for i in range(16):
        for j in range(16):
            D[i,j] = C[i,j]
            for k in range(16):
                D[i,j] += A[i,k]*B[k,j]
            
    return D

# NOTE: MU only accepts single-precision square matrices of size 16.

# Example usage (also for first-time JIT compiling)
A = np.random.randn(16,16).astype(np.float32)
B = np.random.randn(16,16).astype(np.float32)
C = np.random.randn(16,16).astype(np.float32)

D = MU(A,B,C)

dtype = np.float32

m = 32
n = 16
k = 1024

M = 16

A = np.random.randn(m,k).astype(dtype)
B = np.random.randn(k,n).astype(dtype)

Solution to Question 5
----------------------

In [9]:
# Example usage (also for first-time JIT compiling)
A = np.random.randn(16,16).astype(np.float32)
B = np.random.randn(16,16).astype(np.float32)
C = np.random.randn(16,16).astype(np.float32)

D = MU(A,B,C)
assert abs(D - (A@B + C)).max() < 1.0e-5

############### Solution ################

dtype = np.float32

m = 32
n = 16
k = 1024

M = 16

A = np.random.randn(m,k).astype(dtype)
B = np.random.randn(k,n).astype(dtype)

def matmat(A,B):
    kblocks = k//M
    mblocks = m//M
    nblocks = n//M

    out = np.zeros((m,n), dtype=dtype)

    for ii in range(mblocks):
        for jj in range(nblocks):
            for kk in range(kblocks):
                out[ii*M:(ii+1)*M, jj*M:(jj+1)*M] = MU(A[ii*M:(ii+1)*M, kk*M:(kk+1)*M], B[kk*M:(kk+1)*M, jj*M:(jj+1)*M], out[ii*M:(ii+1)*M, jj*M:(jj+1)*M])

    return out

D = matmat(A,B)
Dex = A@B

relerr = abs(D - Dex).max()/abs(Dex).max()

assert relerr < 1.0e-5

Question 6 (Optional)
=====================

This question is optional. Only work on this if you are done with everything else and you still have time.

Can you compute the peak performance (GFLOPS) of [Intel AMX-BF16 units](https://en.wikipedia.org/wiki/Advanced_Matrix_Extensions) VS [Intel AVX512-BF16 units](https://en.wikipedia.org/wiki/AVX-512#BF16) assuming the CPU frequency is 2 GHz? Use the hyperlinks and Google to find the information required.